# Eflux2 applied to *S. elongatus*

In [6]:
import sys
sys.path.append('../src')
sys.path.append('/Users/mahs128/Repos/CONCERTO')
from concerto.helpers.load_model_from_git import load_model_from_git
from eflux2 import EFlux2
import cobra
import pandas as pd

## Load in inputs

### Read in transcriptomics data

In [14]:
transcriptomics_fname = "processed_data/cleaned_transcriptomics.csv"
transcriptomics_df = pd.read_csv(transcriptomics_fname, index_col="Label")
transcriptomics_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Label,,,,,,,,,
SYNPCC7942_RS00005,16290,17761,15101,14567,13967,12653,7016,8855,8689
SYNPCC7942_RS00010,7185,7502,6565,10086,7546,7705,3747,6670,6960
SYNPCC7942_RS00015,24176,26304,22781,23926,21306,20412,13440,17253,17053
SYNPCC7942_RS00020,35119,35145,25895,37701,34175,30569,24835,39769,30280
SYNPCC7942_RS00025,6891,7803,6607,6347,6284,6844,3326,4146,3180


### Load SBML model

In [23]:
syn_model = load_model_from_git('Synechococcus')

### Load rates

In [9]:
### TODO: MISSING RATES FOR SYN MODEL ###

## Normalize relevant data with respect to reference strain/line

### Identify reference strain/line

In [21]:
ref_day_rep = transcriptomics_df.columns[4] # 'Se_axen_d6_2'

### Normalize transcriptomics data

In [22]:
normalized_transcriptomics_df = transcriptomics_df.div(transcriptomics_df[ref_day_rep], axis=0)
normalized_transcriptomics_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Label,,,,,,,,,
SYNPCC7942_RS00005,1.166321,1.271640,1.081191,1.042958,1.0,0.905921,0.502327,0.633994,0.622109
SYNPCC7942_RS00010,0.952160,0.994169,0.869997,1.336602,1.0,1.021071,0.496554,0.883912,0.922343
SYNPCC7942_RS00015,1.134704,1.234582,1.069229,1.122970,1.0,0.958040,0.630808,0.809772,0.800385
SYNPCC7942_RS00020,1.027623,1.028383,0.757718,1.103175,1.0,0.894484,0.726701,1.163687,0.886028
SYNPCC7942_RS00025,1.096595,1.241725,1.051400,1.010025,1.0,1.089115,0.529281,0.659771,0.506047


In [18]:
### TODO: Normalize rates ?????

## Calculate fluxes

### Explore CO2 exchange and transport from external compartment --> periplasm --> cytoplasm


In [39]:
print("----")
print("co2_e reactions")
print("----")
for r in syn_model.metabolites.co2_e.reactions:
    print(r.id)

print(" ")
print("----")
print("co2_p reactions")
print("----")
for r in syn_model.metabolites.co2_p.reactions:
    print(r.id)



----
co2_e reactions
----
EX_co2_e
CO2tex
 
----
co2_p reactions
----
CO2tex
CO2tpp


In [ ]:
display(syn_model.reactions.get_by_id('EX_co2_e'))
display(syn_model.reactions.get_by_id('CO2tex'))
display(syn_model.reactions.get_by_id('CO2tpp')) 

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x2a2087880
Stoichiometry,co2_e <=> CO2 CO2 <=>
GPR,
Lower bound,-1.99
Upper bound,1000.0


Reaction identifier,CO2tex
Name,CO2 transport via diffusion (extracellular to periplasm)
Memory address,0x2a20598b0
Stoichiometry,co2_e <=> co2_p CO2 CO2 <=> CO2 CO2
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Reaction identifier,CO2tpp
Name,CO2 transporter via diffusion (periplasm)
Memory address,0x17f709c70
Stoichiometry,co2_p <=> co2_c CO2 CO2 <=> CO2 CO2
GPR,
Lower bound,-1000.0
Upper bound,1000.0


CO2tex and CO2tpp are possible candidates for CO2 uptake reaction targets.
However they both occur through diffusion, which isn't necessarily attributed to a gene/regulator.
Therefore, need to take a closer look at cytoplasmic reactions, co2_c

### Explore CO2 internal reactions taking place in the cytoplasm

In [47]:
print("----")
print("co2_c reactions")
print("----")
for r in syn_model.metabolites.co2_c.reactions:
    print(r.reaction.split('-->'))

----
co2_c reactions
----
['h_c + thrp_c ', ' applp_c + co2_c']
['h_c + malACP_c + palmACP_c ', ' 3ooctdACP_c + ACP_c + co2_c']
['3nphb_c + amet_c + h2o_c + nad_c ', ' 2m6sbenzq_c + ahcys_c + co2_c + h_c + nadh_c']
['2.0 for_c + 2.0 h_c + o2_c ', ' 2.0 co2_c + 2.0 h2o_c']
['Ncbmpts_c + h2o_c + 2.0 h_c ', ' co2_c + nh4_c + ptrc_c']
['2cpr5p_c + h_c ', ' 3ig3p_c + co2_c + h2o_c']
['2.0 h_c + prpp_c + quln_c ', ' co2_c + nicrnt_c + ppi_c']
['h_c + orn_c ', ' co2_c + ptrc_c']
['arg__L_c + h_c ', ' agm_c + co2_c']
['gmeACP_c + h_c + malACP_c ', ' ACP_c + co2_c + opmeACP_c']
['dcaACP_c + h_c + malACP_c ', ' 3oddecACP_c + ACP_c + co2_c']
['co2_c + 2.0 fdxrd_c + h2o_c + 4.0 h_c + pq_um ', ' 2.0 fdxox_c + 3.0 h_u + hco3_c + pqh2_um']
['akg_c + o2_c + pro__L_c ', ' 4hpro__LT_c + co2_c + succ_c']
['2a3pp_c + h_c ', ' 3a2oxpp_c + co2_c']
['h_c + pyr_c + thmpp_c ', ' 2ahethmpp_c + co2_c']
['6pgc_c + nadp_c ', ' co2_c + nadph_c + ru5p__D_c']
['2mahmp_c + cthzp_c + 2.0 h_c ', ' co2_c + ppi_c + thmmp_

In [41]:
# Explore sucrose exchange and transport from cytoplasm --> external compartment
print("----")
print("sucr_c reactions")
print("----")
for r in syn_model.metabolites.sucr_c.reactions:
    print(r.id)

print(" ")
print("----")
print("sucr_e reactions")
print("----")
for r in syn_model.metabolites.sucr_e.reactions:
    print(r.id)

----
sucr_c reactions
----
SUCRt2
SPP
SUCR
 
----
sucr_e reactions
----
SUCRt2
EX_sucr_e


In [42]:
display(syn_model.reactions.get_by_id('SUCR'))
display(syn_model.reactions.get_by_id('SUCRt2'))
display(syn_model.reactions.get_by_id('SPP'))
display(syn_model.reactions.get_by_id('EX_sucr_e')) # candidate for secretion target

Reaction identifier,SUCR
Name,Sucrose hydrolyzing enzyme
Memory address,0x2a22c8df0
Stoichiometry,h2o_c + sucr_c --> fru_c + glc__D_c H2O H2O + Sucrose C12H22O11 --> D-Fructose + D-Glucose
GPR,SYNPCC7942_RS02025
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,SUCRt2
Name,Sucrose transport in via proton symport
Memory address,0x2a2444970
Stoichiometry,h_e + sucr_e <=> h_c + sucr_c H+ + Sucrose C12H22O11 <=> H+ + Sucrose C12H22O11
GPR,cscB
Lower bound,-1000.0
Upper bound,1000.0


Reaction identifier,SPP
Name,Sucrose-phosphate phosphatase
Memory address,0x2a21543d0
Stoichiometry,h2o_c + suc6p_c --> pi_c + sucr_c H2O H2O + Sucrose 6-phosphate --> Phosphate + Sucrose C12H22O11
GPR,SYNPCC7942_RS04180
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,EX_sucr_e
Name,Sucrose C12H22O11 exchange
Memory address,0x2a2406970
Stoichiometry,sucr_e --> Sucrose C12H22O11 -->
GPR,
Lower bound,0.011
Upper bound,1000.0


In [ ]:
intake_target_met = "CO2"/"HCO3" # carbon/carbon dioxide OR bicarbonate???
intake_target_rxn = ""
secrete_target_met = "SUCR" # sucrose
secrete_target_rxn = 'EX_sucr_e'

fluxes = {}
for rep in normalized_transcriptomics_df.columns:
    with syn_model:
        # Collect rates for this replicate/strain/line
        intake_target = syn_model.reactions.id....[co2_intake] # glucose_uptake = normalized_line_rates.loc[rep,'glucose_uptake_rates (mmol/gDCW * hr)' ]
        secrete_target = syn_model.reactions.get_by_id(secrete_target_rxn) # secrete_3hp = normalized_line_rates.loc[rep,'3hp_secretion_rates (mmol/gDCW * hr)']
        # Update model reaction bounds with experimentally calculated rate
        if intake_target:
            getattr(syn_model.reactions, intake_target_rxn).reaction.upper_bound = intake_target
        if secrete_target:
            getattr(syn_model.reactions, secrete_target_rxn).reaction.lower_bound = secrete_target
        try:
            print(rep)
            fluxes[rep] = EFlux2(syn_model, normalized_transcriptomics_df[rep])
        except TypeError:
            print(f"Replicate {rep} with uptake of {intake_target_met} and secretion of {secrete_target_met} is infeasible")
fluxes[rep]